### 모델링을 위한 기본 전처리

- 기본적인 전처리는 파이썬에서 가능한 부분은 파이썬에서 수행하며, SQL에서 가능한 부분은 SQL에서 수행한다.


- 다루는 내용
    - `시간 관련 전처리`
        - DATETIME에서 MONTH, HOUR, WEEKDAY 등 추출
    - `BigQuery GIS`로 Reverse Geocoding
        - 좌표 => zip_code로 변환
    - `학습/ 테스트 데이터 분리`
        - train_split_test
    - `카테고리컬` 데이터 전처리
        - One Hot Encoding
        - Label Encoding

In [1]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import warnings
import seaborn as sns
from sklearn.linear_model import LinearRegression
warnings.filterwarnings('ignore')

PROJECT_ID = 'new-york-taxi-309501'

### 시간관련 전처리
- Bigquery vs Python
    - Bigquery
        - DATETIME_TRUNC 함수를 사용하면 데이터 자르기 가능
        - EXTRACT 함수를 사용하면 MONTH, WEEK, DAY, HOUR등을 쉽게 추출 가능
        - FORMAT_DATETIME 함수를 사용하면 요일 추출 가능( 1이 월요일, Pandas에서는 0이 월요일 )

In [2]:
extract_query = """
SELECT
    *,
    EXTRACT(MONTH FROM pickup_hour) AS month,
    EXTRACT(DAY FROM pickup_hour) AS day,
    CAST(format_datetime('%u', pickup_hour) AS INT64) -1 AS weekday, # Python과 맞추기 위해 -1
    EXTRACT(HOUR FROM pickup_hour) AS hour,
    CASE WHEN CAST(FORMAT_DATETIME('%u', pickup_hour) AS INT64) IN (6, 7) THEN 1 ELSE 0 END AS is_weekend
FROM (
    SELECT 
        DATETIME_TRUNC(pickup_datetime, hour) AS pickup_hour,
        count(*) AS cnt
    FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2015` 
    WHERE EXTRACT(MONTH from pickup_datetime) = 1
    GROUP BY pickup_hour
)
ORDER BY pickup_hour
"""

df = pd.read_gbq(query=extract_query, dialect='standard', project_id='new-york-taxi-309501')

Downloading: 100%|███████████████████████████████████████████████████████████████| 744/744 [00:00<00:00, 1881.15rows/s]
